### Notebook used to generate list of congested branches: read requirements from input table, then generate desired output and copy to accessible location. The output file is used as input to the simulation, which has the ability to change branch properties from inside Matpower.

In [1]:
import os
import numpy as np
import pandas as pd

from postreise.analyze import transmission
from postreise.process import transferdata as td
from westernintnet.westernintnet import win_data

c:\program files\python\python36\lib\site-packages\matplotlib\__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Done loading


* Read info file: table defines what input files go into the branch filter process and what outputfilenames should be
* Note that the File_Location can also be where files can be accessed by other programs, and may not be same as datadir


In [2]:
df = pd.read_csv("../data/TransmissionCongestionInfo.csv", header=0, index_col = 'ID')

In [3]:
df.head()

,Source_Scenario_ID,Congestion_Model,Model_Region,Selected_Region,File_Location,Filename
ID,,,,,,
1,california2020Test01,Binomial,WECC,CA,L/RenewableEnergy/EnergyGridModeling/Data/West...,congestion_CA_only_california2020Test01_zscore...
2,california2030Test01,Binomial,WECC,CA,L/RenewableEnergy/EnergyGridModeling/Data/West...,congestion_CA_only_california2030Test01_zscore...


In [4]:
# Filter by zone; sample - here, filter by CA
# To do: generalize to any zone; needs a lookup table for region-loadzone designation.

CA_zones = [3,4,5,6,7]
CA_buses = win_data.bus.loc[win_data.bus['area'].isin(CA_zones)]
CA_branches = win_data.branches.loc[(win_data.branches['tbus'].isin(CA_buses['busID'])) & 
                                    (win_data.branches['fbus'].isin(CA_buses['busID']))]

fix_list = df.Source_Scenario_ID.tolist()

In [5]:
congestion_dfs = {}
CA_fix_congestion = {}

# To do: Currently, i and s are interchangeable indexes; 
# eventually, s may be used as source scenario for other outputs

for i in df.index:
    print(i)
    scenario = df.loc[i]['Source_Scenario_ID']
    congestion_source = "../data/congestion_stats_" + scenario
    print("Reading %s.csv" % congestion_source)
    try:
        congestion_df = pd.read_csv(congestion_source + '.csv', index_col=0)
    except FileNotFoundError:
        print('Congestion file not found. Will generate it.')
        od = td.OutputData()
        pf = od.get_data(scenario, 'PF')
        win_data.pf_base = pf
        win_data.branches.index = pf.columns

        win_data.branches.loc[win_data.branches.rateA != 0, 'Capacity'] = win_data.branches['rateA']
        win_data.branches.loc[win_data.branches.rateA == 0, 'Capacity'] = 99999.

        win_data.congestion_base = win_data.pf_base.div(win_data.branches['Capacity']).apply(np.abs)  

        congestion_df = transmission.generate_cong_stats(win_data.congestion_base, 
                                                         win_data.branches,
                                                         congestion_source)

    
    congestion_dfs[scenario] = congestion_df  
    CA_fix_congestion[scenario] = congestion_dfs[scenario].loc[
        (congestion_dfs[scenario].index.isin(CA_branches.index)) & 
        (congestion_dfs[scenario]['zscore'] >=1)]
    
    filename = '../data/' + df.loc[i]['Filename']  
    
    # To write in the group-accessible folder
    # filename = df.loc[i]['File_Location'] + df.loc[i]['Filename']  

    if not os.path.exists(filename):
        CA_fix_congestion[scenario].to_csv(filename)
                
    print(scenario, len(CA_fix_congestion[scenario]))

1
Reading ../data/congestion_stats_california2020Test01.csv
california2020Test01 51
2
Reading ../data/congestion_stats_california2030Test01.csv
california2030Test01 129
